In [1]:
%load_ext autoreload
%autoreload 2

In [140]:
import time 

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pinocchio as pin
from scipy.optimize import least_squares
from scipy.special import huber

from projectyl.dynamics.armmodel import ArmRobot
from projectyl.dynamics.meshcat_viewer_wrapper import MeshcatVisualizer

In [141]:
arm_robot = ArmRobot(upper_arm_length=0.3, forearm_length=0.25)

In [142]:
viz = MeshcatVisualizer(arm_robot)
arm_robot.model.createData()
viz.display(arm_robot.q0)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


TESTS arm moving, random constant acceleration

In [163]:
q = arm_robot.q0.copy()
vq = np.zeros(arm_robot.model.nv)
aq = 5 * np.random.rand(arm_robot.model.nv)
viz.display(q)
DT = 1e-2

T = 30

rec_gt_sol = []
rec_gt_p = []

shoulder_joint_id = arm_robot.model.getJointId("shoulder")
elbow_joint_id = arm_robot.model.getJointId("elbow")
end_effector_frame_id = arm_robot.model.getFrameId("end_effector")

In [164]:
for _ in range(T):
    vq += aq * DT
    q = pin.integrate(arm_robot.model, q, vq * DT)
    tauq = pin.rnea(arm_robot.model, arm_robot.data, q, vq, aq)

    pin.framesForwardKinematics(arm_robot.model, arm_robot.data, q)
    shoulder_p = arm_robot.data.oMi[shoulder_joint_id].translation
    elbow_p = arm_robot.data.oMi[elbow_joint_id].translation
    end_effector_p = arm_robot.data.oMf[end_effector_frame_id].translation

    viz.display(q)

    xuc = np.concatenate([q, vq, tauq])
    rec_gt_sol.append(xuc)

    p = np.concatenate([shoulder_p, elbow_p, end_effector_p])
    rec_gt_p.append(p)
    time.sleep(1e-2)

In [165]:
gt_sol = np.vstack(rec_gt_sol)
gt_p = np.vstack(rec_gt_p)

In [166]:
def get_pose_velocity_from_state(tq_normalized, tvq, T):
    rec_p = []
    rec_v = []

    for t in range(T):
        q = tq_normalized[t]
        vq = tvq[t]
        
        pin.forwardKinematics(arm_robot.model, arm_robot.data, q, vq)
        pin.updateFramePlacements(arm_robot.model, arm_robot.data)
        
        shoulder_p = arm_robot.data.oMi[shoulder_joint_id].translation
        elbow_p = arm_robot.data.oMi[elbow_joint_id].translation
        end_effector_p = arm_robot.data.oMf[end_effector_frame_id].translation

        shoulder_lv = arm_robot.data.v[shoulder_joint_id]
        elbow_lv = arm_robot.data.v[elbow_joint_id]
        
        shoulder_v = arm_robot.data.oMi[shoulder_joint_id].act(shoulder_lv).linear
        elbow_v = arm_robot.data.oMi[elbow_joint_id].act(elbow_lv).linear
        end_effector_v = pin.getFrameVelocity(arm_robot.model, arm_robot.data, end_effector_frame_id, pin.ReferenceFrame.WORLD).linear

        p = np.concatenate([shoulder_p, elbow_p, end_effector_p])
        rec_p.append(p)

        v = np.concatenate([shoulder_v, elbow_v, end_effector_v])
        rec_v.append(v)

    tp = np.vstack(rec_p)
    tv = np.vstack(rec_v)

    ta = (tv[1:] - tv[:-1]) / DT

    return tp, tv, ta

In [167]:
def diff_3D(tp, tp_observed):
    diff = tp - tp_observed
    diff = diff.flatten()

    return diff

In [168]:
def smooth_velocity_acceleration(tv, ta):

    rv = tv.flatten()
    ra = ta.flatten()

    return np.concatenate([rv, ra])

In [169]:
def smooth_torque(ttauq):
    return ttauq.flatten()

In [170]:
def full_body_dynamics(tq_normalized, tvq, taq, ttauq, T):
    
    tau_rec = []

    for t in range(1, T):
        tau = pin.rnea(arm_robot.model, arm_robot.data, tq_normalized[t], tvq[t], taq[t - 1])

        tau_rec.append(tau)
    
    ttau = np.vstack(tau_rec)

    diff = ttau - ttauq[1:]
    diff = diff.flatten()

    return diff

In [171]:
def objective(txuc, tp_observed, T, debug=False):
    txuc_r = txuc.reshape(T, -1)

    tq = txuc_r[:, :arm_robot.model.nq]
    norm_quat = np.linalg.norm(tq[:, :-1], axis=1, keepdims=True)
    tq_normalized = tq.copy()
    tq_normalized[:, :-1] /= norm_quat

    tvq = txuc_r[:, arm_robot.model.nq: arm_robot.model.nq + arm_robot.model.nv]

    taq = (tvq[1:] - tvq[:-1]) / DT

    ttauq = txuc_r[:, arm_robot.model.nq + arm_robot.model.nv:]

    tp, tv, ta = get_pose_velocity_from_state(tq_normalized, tvq, T)

    # DEBUG
    if debug:
        print("3D joint positions", np.linalg.norm(diff_3D(tp, tp_observed)))
        print("Smooth velocity acceleration", np.linalg.norm(smooth_velocity_acceleration(tv, ta)))
        print("Smooth torque", np.linalg.norm(smooth_torque(ttauq)))
        print("Full body dynamics", np.linalg.norm(full_body_dynamics(tq_normalized, tvq, taq, ttauq, T)))

    res = np.concatenate([
        diff_3D(tp, tp_observed),
        0.1 * smooth_velocity_acceleration(tv, ta),
        0.1 * smooth_torque(ttauq),
        10 * full_body_dynamics(tq_normalized, tvq, taq, ttauq, T),
    ])

    return res

In [172]:
q = gt_sol[:, :arm_robot.model.nq]

initial = np.concatenate([q, np.zeros((T, 2 * arm_robot.model.nv))], axis=1)

In [173]:
print(initial.shape)

(30, 13)


In [174]:
print(gt_sol.shape)

(30, 13)


In [175]:
sol = least_squares(objective, initial.flatten(), args=(gt_p, T)).x

In [176]:
print(np.linalg.norm(gt_sol.flatten()- sol.flatten()))

29.213940625183607


In [177]:
res = sol.reshape(T, -1)

In [178]:
np.linalg.norm(objective(sol, gt_p, T, True))

3D joint positions 0.022816546794363162
Smooth velocity acceleration 0.11207147386582664
Smooth torque 0.9375185324955524
Full body dynamics 9.375156004570234e-05


0.0971415661979525

In [179]:
np.linalg.norm(objective(gt_sol.flatten(), gt_p, T, True))

3D joint positions 1.6697012885238947e-16
Smooth velocity acceleration 32.961498482327684
Smooth torque 27.680191993514953
Full body dynamics 7.281050273043157e-15


4.304246055929354

In [182]:
tq = res[:, :arm_robot.model.nq]
print(tq.shape)
norm_quat = np.linalg.norm(tq[:, :-1], axis=1, keepdims=True)
tq_normalized = tq.copy()
tq_normalized[:, :-1] /= norm_quat

tvq = res[:, arm_robot.model.nq: arm_robot.model.nq + arm_robot.model.nv]


ttauq = res[:, arm_robot.model.nq + arm_robot.model.nv:]

(30, 5)


In [201]:
for t in range(T):

    viz.display(tq_normalized[t])

    time.sleep(1e-2)

In [202]:
for t in range(T):
    viz.display(gt_sol[t, :arm_robot.model.nq])
    time.sleep(1e-2)